# Scrape human made images

This code is designed to scrape images from Google search results. It utilizes Selenium and Chromedriver to automate the process of visiting search result pages, scrolling down to load more images, and extracting the image URLs. The images are then downloaded and saved into specific category folders.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request
import ssl

# Set up directories
HUMAN = '/Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human'

# Specify the path to the ChromeDriver
CHROME_DRIVER_PATH = '/Users/sakai/Downloads/chromedriver-mac-arm64-2/chromedriver'


# URLs to visit
urls = [
       'https://www.google.com/search?q=people+outside&tbm=isch&ved=2ahUKEwid5LC33pr-AhX0HzQIHWLOAkMQ2-cCegQIABAA&oq=people+outside&gs_lcp=CgNpbWcQAzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6BwgAEIoFEEM6CggAEIoFELEDEEM6CAgAEIAEELEDOggIABCxAxCDAToGCAAQCBAeUMsIWNkrYJUsaARwAHgAgAFtiAHPDJIBBDE2LjKYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=tZoxZN21H_S_0PEP4pyLmAQ&bih=609&biw=1280',
     ]

# Categories for images
categories = [
     'c_people_outside'
]

# Bỏ qua SSL verification
context = ssl._create_unverified_context()

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Runs Chrome in headless mode.
chrome_service = Service(CHROME_DRIVER_PATH)

# Initialize the browser driver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Tải và lưu ảnh từ mỗi URL
for idx, url in enumerate(urls):
    driver.get(url)
    time.sleep(2)  # Đợi trang tải xong

    # Cuộn trang để tải ảnh
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Tìm tất cả các ảnh sau khi cuộn
    images = driver.find_elements(By.TAG_NAME, 'img')
    os.makedirs(os.path.join(HUMAN, categories[idx]), exist_ok=True)

    # Duyệt và tải từng ảnh
    for i, img in enumerate(images):
        src = img.get_attribute('src')

        # Kiểm tra thêm thuộc tính 'data-src' nếu 'src' rỗng
        if not src:
            src = img.get_attribute('data-src')

        if src and 'http' in src:
            try:
                file_path = os.path.join(HUMAN, categories[idx], f'image_{i}.jpg')

                # Tải ảnh với urlopen và bỏ qua xác minh SSL
                with urllib.request.urlopen(src, context=context) as response, open(file_path, 'wb') as out_file:
                    out_file.write(response.read())

                print(f"Đã tải xong ảnh {i} từ '{categories[idx]}' - Lưu tại: {file_path}")

            except Exception as e:
                print(f"Lỗi khi tải ảnh {i}: {e}")

driver.quit()


Đã tải xong ảnh 0 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_0.jpg
Đã tải xong ảnh 12 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_12.jpg
Đã tải xong ảnh 13 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_13.jpg
Đã tải xong ảnh 54 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_54.jpg
Đã tải xong ảnh 56 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_56.jpg
Đã tải xong ảnh 58 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_58.jpg
Đã tải xong ảnh 60 từ 'c_people_outside' - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human/c_people_outside/image_60.jpg
Đã tải xong ảnh 62 từ 'c_people_outside' - Lưu tại

# Scrape DALLE images

This code is used to scrape images from the website dalle2.gallery, which hosts AI-generated images created by DALLE. It automates the process of visiting the website, scrolling down to load more images, and downloading the unique image URLs until the specified maximum number of images (max_images) is reached. The downloaded images are saved with sequential numbers as filenames.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request
import ssl

# Specify the path to the ChromeDriver
CHROME_DRIVER_PATH = '/Users/sakai/Downloads/chromedriver-mac-arm64-2/chromedriver'
urls = ['https://dalle2.gallery/#search-random']
DALLE = '/Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE'

# Bỏ qua SSL verification
context = ssl._create_unverified_context()

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Runs Chrome in headless mode.
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-insecure-localhost')
chrome_options.add_argument('--disable-web-security')
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_service = Service(CHROME_DRIVER_PATH)

# Initialize the browser driver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# # Tải và lưu ảnh từ mỗi URL
for idx, url in enumerate(urls):
    driver.get(url)
    print('tai trang xong')
    time.sleep(2)  # Đợi trang tải xong

    # Cuộn trang để tải ảnh
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(50):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Tìm tất cả các ảnh sau khi cuộn
    images = driver.find_elements(By.TAG_NAME, 'img')

    print(len(images))

    # Duyệt và tải từng ảnh
    for i, img in enumerate(images):
        src = img.get_attribute('src')
        
        print(f'duyet va tai tung anh {i}')

        # Kiểm tra thêm thuộc tính 'data-src' nếu 'src' rỗng
        if not src:
            src = img.get_attribute('data-src')

        if src and 'http' in src:
            try:
                file_path = os.path.join(DALLE, f'image_{i}.jpg')

                # Tải ảnh với urlopen và bỏ qua xác minh SSL
                with urllib.request.urlopen(src, context=context) as response, open(file_path, 'wb') as out_file:
                    out_file.write(response.read())

                print(f"Đã tải xong ảnh {i} - Lưu tại: {file_path}")

            except Exception as e:
                print(f"Lỗi khi tải ảnh {i}: {e}")

driver.quit()

tai trang xong
190
duyet va tai tung anh 0
Đã tải xong ảnh 0 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_0.jpg
duyet va tai tung anh 1
Đã tải xong ảnh 1 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_1.jpg
duyet va tai tung anh 2
Đã tải xong ảnh 2 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_2.jpg
duyet va tai tung anh 3
Đã tải xong ảnh 3 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_3.jpg
duyet va tai tung anh 4
Đã tải xong ảnh 4 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_4.jpg
duyet va tai tung anh 5
Đã tải xong ảnh 5 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_5.jpg
duyet va tai tung anh 6
Đã tải xong ảnh 6 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_6.jpg
duyet va tai tung anh 7
Đã tải xong ảnh 7 - Lưu tại: /Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE/image_7.jpg
duyet va tai tung anh 8
Đã tả